<a href="https://colab.research.google.com/github/Jorge-89/Regresiones_TP_UNSAM/blob/main/SVM_Regresion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importo librerias necesarias
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error as mse
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [2]:
#se importa el set de datos
url = 'https://raw.githubusercontent.com/Jorge-89/Regresion-Lineal-/main/base_datos_estaciones_met_V3_train.csv'
df = pd.read_csv(url, sep=",")
df.columns

Index(['Unnamed: 0', 'fecha_completa', 'Fecha', 'Hora',
       'Temp_Alicia AgriculturaCba', 'Humedad_Alicia AgriculturaCba',
       'PP_Alicia AgriculturaCba', 'Temp_Las Varas AgriculturaCba',
       'Humedad_Las Varas AgriculturaCba', 'PP_Las Varas AgriculturaCba',
       'Temp_San Miguel - Establecimiento Don Luis',
       'Humedad_San Miguel - Establecimiento Don Luis',
       'PP_San Miguel - Establecimiento Don Luis',
       'Temp_San Miguel - Listello', 'Humedad_San Miguel - Listello',
       'PP_San Miguel - Listello', 'Temp_ San Miguel - Las Varillas',
       'Humedad_ San Miguel - Las Varillas', 'PP_ San Miguel - Las Varillas'],
      dtype='object')

In [3]:
x= df[[ 'Temp_Las Varas AgriculturaCba', 'Humedad_Las Varas AgriculturaCba',
        'Temp_San Miguel - Establecimiento Don Luis', 'Humedad_San Miguel - Establecimiento Don Luis',
       'Temp_San Miguel - Listello', 'Humedad_San Miguel - Listello',
       'Temp_ San Miguel - Las Varillas', 'Humedad_ San Miguel - Las Varillas']]
y= df["Temp_Alicia AgriculturaCba"]

In [4]:
#Separo los datos de "train" en entrenamiento y prueba "test" para probar los algoritmos
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
 from sklearn.model_selection import RandomizedSearchCV

In [8]:
modelo= Pipeline([('scaler', StandardScaler()),
                    
                  ('model',SVR())
])

grilla = RandomizedSearchCV(modelo,
                      [{ 'model__C':[1,10,300,500,1000],'model__kernel':['linear']},{'model__gamma':[0.01,0.1,0.5,1], 'model__C':[10,300,500,1000],'model__kernel':['rbf']}],
                      cv = 5, scoring="neg_mean_squared_error",verbose=3,n_jobs=-1
                      )
grilla.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  8.9min finished


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('model', SVR())]),
                   n_jobs=-1,
                   param_distributions=[{'model__C': [1, 10, 300, 500, 1000],
                                         'model__kernel': ['linear']},
                                        {'model__C': [10, 300, 500, 1000],
                                         'model__gamma': [0.01, 0.1, 0.5, 1],
                                         'model__kernel': ['rbf']}],
                   scoring='neg_mean_squared_error', verbose=3)

In [23]:
#El mejor modelo fue rbf, asi que optmizo esos parametros.
modelo= Pipeline([('scaler', StandardScaler()),
                    
                  ('model',SVR())
])

grilla = GridSearchCV(modelo,
                      {'model__gamma':[2,2.5,3], 'model__C':[1,10,50,100],'model__kernel':['rbf']},
                      cv = 5, scoring="neg_mean_squared_error",verbose=3, n_jobs=-1
                      )
grilla.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   22.9s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:  5.3min finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', SVR())]),
             n_jobs=-1,
             param_grid={'model__C': [1, 10, 50, 100],
                         'model__gamma': [2, 2.5, 3],
                         'model__kernel': ['rbf']},
             scoring='neg_mean_squared_error', verbose=3)

In [24]:
grilla.best_params_

{'model__C': 50, 'model__gamma': 2.5, 'model__kernel': 'rbf'}

In [25]:
grilla.best_score_

-0.3263362727719414

In [26]:
# Fit best model
modelo=grilla.best_estimator_
modelo.fit(x_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('model', SVR(C=50, gamma=2.5))])

In [27]:
y_pred_test = modelo.predict(x_test)
y_pred_test

array([24.67905355, 27.23948158, 14.66453937, ..., 33.77864068,
       26.61673815, 29.20816322])

In [28]:
print('Precisión del modelo usando el (MSE):')
score_testeo = mse(y_test,y_pred_test)

print (score_testeo)

Precisión del modelo usando el (MSE):
0.2888853999617873


In [38]:

modelo2=Pipeline([['poly',StandardScaler()],['regressor',SVR(kernel='rbf',C=50, gamma=2.5)]])
#entreno el modelo de grado 1 a ver que tanta diferencia hay en testeo con grado 3
modelo2.fit(x_train,y_train)

Pipeline(steps=[('poly', StandardScaler()),
                ['regressor', SVR(C=50, gamma=2.5)]])

In [29]:
#se importa el set de datos final
url = 'https://raw.githubusercontent.com/Jorge-89/Regresiones_TP_UNSAM/main/base_datos_estaciones_met_V3_test.csv'
df = pd.read_csv(url, sep=",")
df.columns


Index(['Unnamed: 0', 'fecha_completa', 'Fecha', 'Hora',
       'Temp_Alicia AgriculturaCba', 'Humedad_Alicia AgriculturaCba',
       'PP_Alicia AgriculturaCba', 'Temp_Las Varas AgriculturaCba',
       'Humedad_Las Varas AgriculturaCba', 'PP_Las Varas AgriculturaCba',
       'Temp_San Miguel - Establecimiento Don Luis',
       'Humedad_San Miguel - Establecimiento Don Luis',
       'PP_San Miguel - Establecimiento Don Luis',
       'Temp_San Miguel - Listello', 'Humedad_San Miguel - Listello',
       'PP_San Miguel - Listello', 'Temp_ San Miguel - Las Varillas',
       'Humedad_ San Miguel - Las Varillas', 'PP_ San Miguel - Las Varillas'],
      dtype='object')

In [30]:

x= df[[ 'Temp_Las Varas AgriculturaCba', 'Humedad_Las Varas AgriculturaCba',
        'Temp_San Miguel - Establecimiento Don Luis', 'Humedad_San Miguel - Establecimiento Don Luis',
       'Temp_San Miguel - Listello', 'Humedad_San Miguel - Listello',
       'Temp_ San Miguel - Las Varillas', 'Humedad_ San Miguel - Las Varillas']]
y= df["Temp_Alicia AgriculturaCba"]

In [31]:

y_pred = modelo.predict(x)
y_pred

array([19.42333824, 19.40354443, 19.62038612, ..., 13.93159208,
       13.91479496, 13.99980188])

In [39]:
y_pred = modelo2.predict(x)
y_pred

array([19.42333824, 19.40354443, 19.62038612, ..., 13.93159208,
       13.91479496, 13.99980188])

In [40]:

print('Precisión del modelo usando el MSE):')
score_testeo = mse(y_pred,y,squared=True,)

print (score_testeo)


Precisión del modelo usando el MSE):
0.1273027084900858


In [33]:
y_pred

array([19.42333824, 19.40354443, 19.62038612, ..., 13.93159208,
       13.91479496, 13.99980188])

In [34]:
y

0       19.7
1       19.4
2       19.1
3       18.9
4       18.8
        ... 
2587    14.4
2588    14.2
2589    14.1
2590    14.0
2591    13.9
Name: Temp_Alicia AgriculturaCba, Length: 2592, dtype: float64